# 2 register device to fiware on Azure AKS

change ${CORE_ROOT} to your path of `core`.

In [ ]:
export CORE_ROOT="${HOME}/core"

change ${PJ_ROOT} to your path of `example-guide-demo`.

In [ ]:
export PJ_ROOT="${HOME}/example-guide-demo"
cd ${PJ_ROOT};pwd

example)
```
/Users/user/example-guide-demo
```

## load environment variables

load from `core`

In [ ]:
source ${CORE_ROOT}/docs/environments/azure_aks/env

load from `example-guide-demo`

In [ ]:
source ${PJ_ROOT}/docs/environments/azure_aks/env

### register users to RabbitMQ

In [ ]:
for e in $(env); do
  if [[ "${e}" =~ ^MQTT__([[:alnum:]_-]+)=([[:alnum:]_-]+)$ ]]; then
    username=${BASH_REMATCH[1]}
    password=${BASH_REMATCH[2]}
    
    kubectl exec rabbitmq-0 -- rabbitmqctl add_user ${username} ${password}
    kubectl exec rabbitmq-0 -- rabbitmqctl set_permissions -p / ${username} ".*" ".*" ".*"
  fi
done

example)
```
Adding user "iotagent" ...
User "iotagent" already exists
command terminated with exit code 70
Setting permissions for user "iotagent" in vhost "/" ...
Adding user "mobile_robot" ...
Setting permissions for user "mobile_robot" in vhost "/" ...
Adding user "comm_robot" ...
Setting permissions for user "comm_robot" in vhost "/" ...
Adding user "dest_led" ...
Setting permissions for user "dest_led" in vhost "/" ...
Adding user "dest_button" ...
Setting permissions for user "dest_button" in vhost "/" ...
```

In [ ]:
kubectl exec rabbitmq-0 -- rabbitmqctl list_users

example)
```
Listing users ...
user	tags
dest_button	[]
dest_led	[]
mobile_robot	[]
comm_robot	[]
guest	[administrator]
iotagent	[]
```

## register comm_robot service

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-ServicePath: ${COMM_ROBOT_SERVICEPATH}" -H "Content-Type: application/json" https://api.${DOMAIN}/idas/ul20/manage/iot/services/ -X POST -d @- <<__EOS__
{
  "services": [
    {
      "apikey": "${COMM_ROBOT_TYPE}",
      "cbroker": "http://orion:1026",
      "resource": "/iot/d",
      "entity_type": "${COMM_ROBOT_TYPE}"
    }
  ]
}
__EOS__

example)
```json
{}
```

### confirm registered service

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${COMM_ROBOT_SERVICEPATH}" https://api.${DOMAIN}/idas/ul20/manage/iot/services/ | jq .

example)
```json
{
  "count": 1,
  "services": [
    {
      "commands": [],
      "lazy": [],
      "attributes": [],
      "_id": "5cd6105e4d78cc000f950ebc",
      "resource": "/iot/d",
      "apikey": "comm_robot",
      "service": "guide_demo",
      "subservice": "/comm_robot",
      "__v": 0,
      "static_attributes": [],
      "internal_attributes": [],
      "entity_type": "comm_robot"
    }
  ]
}
```

## register comm_robot device

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-ServicePath: ${COMM_ROBOT_SERVICEPATH}" -H "Content-Type: application/json" https://api.${DOMAIN}/idas/ul20/manage/iot/devices/ -X POST -d @- <<__EOS__
{
  "devices": [
    {
      "device_id": "${COMM_ROBOT_ID}",
      "entity_name": "${COMM_ROBOT_ID}",
      "entity_type": "${COMM_ROBOT_TYPE}",
      "timezone": "Asia/Tokyo",
      "protocol": "UL20",
      "attributes": [
        {
          "name": "move",
          "type": "string"
        }
      ],
      "transport": "AMQP"
    }
  ]
}
__EOS__

example)
```json
{}
```

### confirm registered device

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${COMM_ROBOT_SERVICEPATH}" https://api.${DOMAIN}/idas/ul20/manage/iot/devices/${COMM_ROBOT_ID}/ | jq .

example)
```json
{
  "device_id": "xperiahello_01",
  "service": "guide_demo",
  "service_path": "/comm_robot",
  "entity_name": "xperiahello_01",
  "entity_type": "comm_robot",
  "transport": "AMQP",
  "attributes": [
    {
      "object_id": "move",
      "name": "move",
      "type": "string"
    }
  ],
  "lazy": [],
  "commands": [],
  "static_attributes": [],
  "protocol": "UL20"
}
```

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${COMM_ROBOT_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/${COMM_ROBOT_ID}/ | jq .

example)
```json
{
  "id": "xperiahello_01",
  "type": "comm_robot",
  "TimeInstant": {
    "type": "ISO8601",
    "value": " ",
    "metadata": {}
  },
  "move": {
    "type": "string",
    "value": " ",
    "metadata": {}
  }
}
```

### register `cygnus-mongo` as a subscriber of comm_robot device

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -i -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-ServicePath: ${COMM_ROBOT_SERVICEPATH}" -H "Content-Type: application/json" https://api.${DOMAIN}/orion/v2/subscriptions/ -X POST -d @- <<__EOS__
{
  "subject": {
    "entities": [{
      "idPattern": "${COMM_ROBOT_ID}",
      "type": "${COMM_ROBOT_TYPE}"
    }],
    "condition": {
      "attrs": ["move"]
    }
  },
  "notification": {
    "http": {
      "url": "http://cygnus-mongo:5050/notify"
    },
    "attrs": ["move"],
    "attrsFormat": "legacy"
  }
}
__EOS__

example)
```
HTTP/1.1 201 Created
content-length: 0
location: /v2/subscriptions/5cd6126adb70c85cdee24f94
fiware-correlator: dc5c6d16-7380-11e9-b740-f2a8c31f77d6
date: Sat, 11 May 2019 00:08:11 GMT
x-envoy-upstream-service-time: 2026
server: envoy
```

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-ServicePath: ${COMM_ROBOT_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/subscriptions/ | jq .

example)
```json
[
  {
    "id": "5cd6126adb70c85cdee24f94",
    "status": "active",
    "subject": {
      "entities": [
        {
          "idPattern": "xperiahello_01",
          "type": "comm_robot"
        }
      ],
      "condition": {
        "attrs": [
          "move"
        ]
      }
    },
    "notification": {
      "timesSent": 1,
      "lastNotification": "2019-05-11T00:08:10.00Z",
      "attrs": [
        "move"
      ],
      "attrsFormat": "legacy",
      "http": {
        "url": "http://cygnus-mongo:5050/notify"
      },
      "lastSuccess": "2019-05-11T00:08:10.00Z",
      "lastSuccessCode": 200
    }
  }
]
```

## register mobile_robot service

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-ServicePath: ${MOBILE_ROBOT_SERVICEPATH}" -H "Content-Type: application/json" https://api.${DOMAIN}/idas/ul20/manage/iot/services/ -X POST -d @- <<__EOS__
{
  "services": [
    {
      "apikey": "${MOBILE_ROBOT_TYPE}",
      "cbroker": "http://orion:1026",
      "resource": "/iot/d",
      "entity_type": "${MOBILE_ROBOT_TYPE}"
    }
  ]
}
__EOS__

example)
```json
{}
```

### confirm registered service

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${MOBILE_ROBOT_SERVICEPATH}" https://api.${DOMAIN}/idas/ul20/manage/iot/services/ | jq .

example)
```json
{
  "count": 1,
  "services": [
    {
      "commands": [],
      "lazy": [],
      "attributes": [],
      "_id": "5cd616b11466750010c61da2",
      "resource": "/iot/d",
      "apikey": "mobile_robot",
      "service": "guide_demo",
      "subservice": "/mobile_robot",
      "__v": 0,
      "static_attributes": [],
      "internal_attributes": [],
      "entity_type": "mobile_robot"
    }
  ]
}
```

## register mobile_robot device

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-ServicePath: ${MOBILE_ROBOT_SERVICEPATH}" -H "Content-Type: application/json" https://api.${DOMAIN}/idas/ul20/manage/iot/devices/ -X POST -d @- <<__EOS__
{
  "devices": [
    {
      "device_id": "${MOBILE_ROBOT_ID}",
      "entity_name": "${MOBILE_ROBOT_ID}",
      "entity_type": "${MOBILE_ROBOT_TYPE}",
      "timezone": "Asia/Tokyo",
      "protocol": "UL20",
      "attributes": [
        {
          "name": "r_state",
          "type": "string"
        },
        {
          "name": "r_mode",
          "type": "string"
        },
        {
          "name": "x",
          "type": "float32"
        },
        {
          "name": "y",
          "type": "float32"
        },
        {
          "name": "theta",
          "type": "float32"
        },
        {
          "name": "voltage",
          "type": "float32"
        },
        {
          "name": "current",
          "type": "float32"
        },
        {
          "name": "charge",
          "type": "float32"
        },
        {
          "name": "capacity",
          "type": "float32"
        },
        {
          "name": "design_capacity",
          "type": "float32"
        },
        {
          "name": "percentage",
          "type": "float32"
        }
      ],
      "commands": [
        {
          "name": "move",
          "type": "string"
        }
      ],
      "transport": "AMQP"
    }
  ]
}
__EOS__

example)
```json
{}
```

### confirm registered device

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${MOBILE_ROBOT_SERVICEPATH}" https://api.${DOMAIN}/idas/ul20/manage/iot/devices/${MOBILE_ROBOT_ID}/ | jq .

example)
```json
{
  "device_id": "rosbot2_01",
  "service": "guide_demo",
  "service_path": "/mobile_robot",
  "entity_name": "rosbot2_01",
  "entity_type": "mobile_robot",
  "transport": "AMQP",
  "attributes": [
    {
      "object_id": "r_state",
      "name": "r_state",
      "type": "string"
    },
    {
      "object_id": "r_mode",
      "name": "r_mode",
      "type": "string"
    },
    {
      "object_id": "x",
      "name": "x",
      "type": "float32"
    },
    {
      "object_id": "y",
      "name": "y",
      "type": "float32"
    },
    {
      "object_id": "theta",
      "name": "theta",
      "type": "float32"
    },
    {
      "object_id": "voltage",
      "name": "voltage",
      "type": "float32"
    },
    {
      "object_id": "current",
      "name": "current",
      "type": "float32"
    },
    {
      "object_id": "charge",
      "name": "charge",
      "type": "float32"
    },
    {
      "object_id": "capacity",
      "name": "capacity",
      "type": "float32"
    },
    {
      "object_id": "design_capacity",
      "name": "design_capacity",
      "type": "float32"
    },
    {
      "object_id": "percentage",
      "name": "percentage",
      "type": "float32"
    }
  ],
  "lazy": [],
  "commands": [
    {
      "object_id": "move",
      "name": "move",
      "type": "string"
    }
  ],
  "static_attributes": [],
  "protocol": "UL20"
}
```

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${MOBILE_ROBOT_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/${MOBILE_ROBOT_ID}/ | jq .

example)
```json
{
  "id": "rosbot2_01",
  "type": "mobile_robot",
  "TimeInstant": {
    "type": "ISO8601",
    "value": " ",
    "metadata": {}
  },
  "capacity": {
    "type": "float32",
    "value": " ",
    "metadata": {}
  },
  "charge": {
    "type": "float32",
    "value": " ",
    "metadata": {}
  },
  "current": {
    "type": "float32",
    "value": " ",
    "metadata": {}
  },
  "design_capacity": {
    "type": "float32",
    "value": " ",
    "metadata": {}
  },
  "move_info": {
    "type": "commandResult",
    "value": " ",
    "metadata": {}
  },
  "move_status": {
    "type": "commandStatus",
    "value": "UNKNOWN",
    "metadata": {}
  },
  "percentage": {
    "type": "float32",
    "value": " ",
    "metadata": {}
  },
  "r_mode": {
    "type": "string",
    "value": " ",
    "metadata": {}
  },
  "r_state": {
    "type": "string",
    "value": " ",
    "metadata": {}
  },
  "theta": {
    "type": "float32",
    "value": " ",
    "metadata": {}
  },
  "voltage": {
    "type": "float32",
    "value": " ",
    "metadata": {}
  },
  "x": {
    "type": "float32",
    "value": " ",
    "metadata": {}
  },
  "y": {
    "type": "float32",
    "value": " ",
    "metadata": {}
  },
  "move": {
    "type": "string",
    "value": "",
    "metadata": {}
  }
}
```

### register `cygnus-mongo` as a subscriber of mobile_robot device

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -i -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-ServicePath: ${MOBILE_ROBOT_SERVICEPATH}" -H "Content-Type: application/json" https://api.${DOMAIN}/orion/v2/subscriptions/ -X POST -d @- <<__EOS__
{
  "subject": {
    "entities": [{
      "idPattern": "${MOBILE_ROBOT_ID}",
      "type": "${MOBILE_ROBOT_TYPE}"
    }],
    "condition": {
      "attrs": ["r_mode", "r_state", "x", "y", "theta", "move_status", "move_info", "voltage", "current", "charge", "capacity", "design_capacity", "percentage"]
    }
  },
  "notification": {
    "http": {
      "url": "http://cygnus-mongo:5050/notify"
    },
    "attrs": ["r_mode", "r_state", "x", "y", "theta", "move_status", "move_info", "voltage", "current", "charge", "capacity", "design_capacity", "percentage"],
    "attrsFormat": "legacy"
  }
}
__EOS__

example)
```
HTTP/1.1 201 Created
content-length: 0
location: /v2/subscriptions/5cd620e1d9d5db2ffb119d16
fiware-correlator: 7beb02c2-7389-11e9-88b9-46b1c9ec8ccd
date: Sat, 11 May 2019 01:09:53 GMT
x-envoy-upstream-service-time: 4
server: envoy
```

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-ServicePath: ${MOBILE_ROBOT_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/subscriptions/ | jq .

example)
```json
[
  {
    "id": "5cd620e1d9d5db2ffb119d16",
    "status": "active",
    "subject": {
      "entities": [
        {
          "idPattern": "rosbot2_01",
          "type": "mobile_robot"
        }
      ],
      "condition": {
        "attrs": [
          "r_mode",
          "r_state",
          "x",
          "y",
          "theta",
          "move_status",
          "move_info",
          "voltage",
          "current",
          "charge",
          "capacity",
          "design_capacity",
          "percentage"
        ]
      }
    },
    "notification": {
      "timesSent": 1,
      "lastNotification": "2019-05-11T01:09:53.00Z",
      "attrs": [
        "r_mode",
        "r_state",
        "x",
        "y",
        "theta",
        "move_status",
        "move_info",
        "voltage",
        "current",
        "charge",
        "capacity",
        "design_capacity",
        "percentage"
      ],
      "attrsFormat": "legacy",
      "http": {
        "url": "http://cygnus-mongo:5050/notify"
      },
      "lastSuccess": "2019-05-11T01:09:53.00Z",
      "lastSuccessCode": 200
    }
  }
]
```

## register dest_button service

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-ServicePath: ${DEST_BUTTON_SERVICEPATH}" -H "Content-Type: application/json" https://api.${DOMAIN}/idas/ul20/manage/iot/services/ -X POST -d @- <<__EOS__
{
  "services": [
    {
      "apikey": "${DEST_BUTTON_TYPE}",
      "cbroker": "http://orion:1026",
      "resource": "/iot/d",
      "entity_type": "${DEST_BUTTON_TYPE}"
    }
  ]
}
__EOS__

example)
```json
{}
```

### confirm registered service

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DEST_BUTTON_SERVICEPATH}" https://api.${DOMAIN}/idas/ul20/manage/iot/services/ | jq .

example)
```json
{
  "count": 1,
  "services": [
    {
      "commands": [],
      "lazy": [],
      "attributes": [],
      "_id": "5cd63442a1fd25000f1a6bae",
      "resource": "/iot/d",
      "apikey": "button",
      "service": "guide_demo",
      "subservice": "/dest_button",
      "__v": 0,
      "static_attributes": [],
      "internal_attributes": [],
      "entity_type": "button"
    }
  ]
}
```

## register dest_button device

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-ServicePath: ${DEST_BUTTON_SERVICEPATH}" -H "Content-Type: application/json" https://api.${DOMAIN}/idas/ul20/manage/iot/devices/ -X POST -d @- <<__EOS__
{
  "devices": [
    {
      "device_id": "${DEST_BUTTON_ID}",
      "entity_name": "${DEST_BUTTON_ID}",
      "entity_type": "${DEST_BUTTON_TYPE}",
      "timezone": "Asia/Tokyo",
      "protocol": "UL20",
      "attributes": [
        {
          "name": "push",
          "type": "string"
        }
      ],
      "transport": "AMQP"
    }
  ]
}
__EOS__

example)
```json
{}
```

### confirm registered device

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DEST_BUTTON_SERVICEPATH}" https://api.${DOMAIN}/idas/ul20/manage/iot/devices/${DEST_BUTTON_ID}/ | jq .

example)
```json
{
  "device_id": "button_01",
  "service": "guide_demo",
  "service_path": "/dest_button",
  "entity_name": "button_01",
  "entity_type": "button",
  "transport": "AMQP",
  "attributes": [
    {
      "object_id": "push",
      "name": "push",
      "type": "string"
    }
  ],
  "lazy": [],
  "commands": [],
  "static_attributes": [],
  "protocol": "UL20"
}
```

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DEST_BUTTON_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/${DEST_BUTTON_ID}/ | jq .

example)
```json
{
  "id": "button_01",
  "type": "button",
  "TimeInstant": {
    "type": "ISO8601",
    "value": " ",
    "metadata": {}
  },
  "push": {
    "type": "string",
    "value": " ",
    "metadata": {}
  }
}
```

### register `cygnus-mongo` as a subscriber of dest_button device

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -i -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-ServicePath: ${DEST_BUTTON_SERVICEPATH}" -H "Content-Type: application/json" https://api.${DOMAIN}/orion/v2/subscriptions/ -X POST -d @- <<__EOS__
{
  "subject": {
    "entities": [{
      "idPattern": "${DEST_BUTTON_ID}",
      "type": "${DEST_BUTTON_TYPE}"
    }],
    "condition": {
      "attrs": ["push"]
    }
  },
  "notification": {
    "http": {
      "url": "http://cygnus-mongo:5050/notify"
    },
    "attrs": ["push"],
    "attrsFormat": "legacy"
  }
}
__EOS__

example)
```
HTTP/1.1 201 Created
content-length: 0
location: /v2/subscriptions/5cd6350bd9d5db2ffb119d17
fiware-correlator: 8077aba4-7395-11e9-88b9-46b1c9ec8ccd
date: Sat, 11 May 2019 02:35:55 GMT
x-envoy-upstream-service-time: 4
server: envoy
```

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-ServicePath: ${DEST_BUTTON_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/subscriptions/ | jq .

example)
```json
[
  {
    "id": "5cd6350bd9d5db2ffb119d17",
    "status": "active",
    "subject": {
      "entities": [
        {
          "idPattern": "button_01",
          "type": "button"
        }
      ],
      "condition": {
        "attrs": [
          "push"
        ]
      }
    },
    "notification": {
      "timesSent": 1,
      "lastNotification": "2019-05-11T02:35:55.00Z",
      "attrs": [
        "push"
      ],
      "attrsFormat": "legacy",
      "http": {
        "url": "http://cygnus-mongo:5050/notify"
      },
      "lastSuccess": "2019-05-11T02:35:55.00Z",
      "lastSuccessCode": 200
    }
  }
]
```

## register dest_led service

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-ServicePath: ${DEST_LED_SERVICEPATH}" -H "Content-Type: application/json" https://api.${DOMAIN}/idas/ul20/manage/iot/services/ -X POST -d @- <<__EOS__
{
  "services": [
    {
      "apikey": "${DEST_LED_TYPE}",
      "cbroker": "http://orion:1026",
      "resource": "/iot/d",
      "entity_type": "${DEST_LED_TYPE}"
    }
  ]
}
__EOS__

example)
```json
{}
```

### confirm registered service

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DEST_LED_SERVICEPATH}" https://api.${DOMAIN}/idas/ul20/manage/iot/services/ | jq .

example)
```json
{
  "count": 1,
  "services": [
    {
      "commands": [],
      "lazy": [],
      "attributes": [],
      "_id": "5cd6390a1466750010c61da3",
      "resource": "/iot/d",
      "apikey": "led",
      "service": "guide_demo",
      "subservice": "/dest_led",
      "__v": 0,
      "static_attributes": [],
      "internal_attributes": [],
      "entity_type": "led"
    }
  ]
}
```

## register dest_led device

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-ServicePath: ${DEST_LED_SERVICEPATH}" -H "Content-Type: application/json" https://api.${DOMAIN}/idas/ul20/manage/iot/devices/ -X POST -d @- <<__EOS__
{
  "devices": [
    {
      "device_id": "${DEST_LED_ID}",
      "entity_name": "${DEST_LED_ID}",
      "entity_type": "${DEST_LED_TYPE}",
      "timezone": "Asia/Tokyo",
      "protocol": "UL20",
      "commands": [
        {
          "name": "action",
          "type": "string"
        }
      ],
      "transport": "AMQP"
    }
  ]
}
__EOS__

example)
```json
{}
```

### confirm registered device

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DEST_LED_SERVICEPATH}" https://api.${DOMAIN}/idas/ul20/manage/iot/devices/${DEST_LED_ID}/ | jq .

example)
```json
{
  "device_id": "led_01",
  "service": "guide_demo",
  "service_path": "/dest_led",
  "entity_name": "led_01",
  "entity_type": "led",
  "transport": "AMQP",
  "attributes": [],
  "lazy": [],
  "commands": [
    {
      "object_id": "action",
      "name": "action",
      "type": "string"
    }
  ],
  "static_attributes": [],
  "protocol": "UL20"
}
```

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DEST_LED_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/${DEST_LED_ID}/ | jq .

example)
```json
{
  "id": "led_01",
  "type": "led",
  "TimeInstant": {
    "type": "ISO8601",
    "value": " ",
    "metadata": {}
  },
  "action_info": {
    "type": "commandResult",
    "value": " ",
    "metadata": {}
  },
  "action_status": {
    "type": "commandStatus",
    "value": "UNKNOWN",
    "metadata": {}
  },
  "action": {
    "type": "string",
    "value": "",
    "metadata": {}
  }
}
```

### register `cygnus-mongo` as a subscriber of dest_led device

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -i -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-ServicePath: ${DEST_LED_SERVICEPATH}" -H "Content-Type: application/json" https://api.${DOMAIN}/orion/v2/subscriptions/ -X POST -d @- <<__EOS__
{
  "subject": {
    "entities": [{
      "idPattern": "${DEST_LED_ID}",
      "type": "${DEST_LED_TYPE}"
    }],
    "condition": {
      "attrs": ["action_info", "action_status"]
    }
  },
  "notification": {
    "http": {
      "url": "http://cygnus-mongo:5050/notify"
    },
    "attrs": ["action_info", "action_status"],
    "attrsFormat": "legacy"
  }
}
__EOS__

example)
```
HTTP/1.1 201 Created
content-length: 0
location: /v2/subscriptions/5cd63ae1d9d5db2ffb119d18
fiware-correlator: fae114a4-7398-11e9-83e5-46b1c9ec8ccd
date: Sat, 11 May 2019 03:00:48 GMT
x-envoy-upstream-service-time: 4
server: envoy
```

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-ServicePath: ${DEST_LED_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/subscriptions/ | jq .

example)
```json
[
  {
    "id": "5cd6350bd9d5db2ffb119d17",
    "status": "active",
    "subject": {
      "entities": [
        {
          "idPattern": "button_01",
          "type": "button"
        }
      ],
      "condition": {
        "attrs": [
          "push"
        ]
      }
    },
    "notification": {
      "timesSent": 1,
      "lastNotification": "2019-05-11T02:35:55.00Z",
      "attrs": [
        "push"
      ],
      "attrsFormat": "legacy",
      "http": {
        "url": "http://cygnus-mongo:5050/notify"
      },
      "lastSuccess": "2019-05-11T02:35:55.00Z",
      "lastSuccessCode": 200
    }
  }
]
```

## test comm_robot device

### prepare a command to subscribe all topics

In [ ]:
echo "mosquitto_sub -h mqtt.${DOMAIN} -p 8883 --cafile ${CORE_ROOT}/secrets/DST_Root_CA_X3.pem -d -u iotagent -P ${MQTT__iotagent} -t /#"

### subscribe all topic

_Outside of this notebook_
1. open a ternminal.
1. run the above command displayed `prepare a command to subscribe all topics`.

### publish a MQTT message to emulate receiving the event from comm_robot

In [ ]:
d=$(date '+%Y-%m-%dT%H:%M:%S.%s+09:00')
mosquitto_pub -h mqtt.${DOMAIN} -p 8883 --cafile ${CORE_ROOT}/secrets/DST_Root_CA_X3.pem -d -u iotagent -P ${MQTT__iotagent} -t /${COMM_ROBOT_TYPE}/${COMM_ROBOT_ID}/attrs -m "${d}|move|dest01"

example)
```
Client mosq/E7sCzVC1BFNFCq27Gh sending CONNECT
Client mosq/E7sCzVC1BFNFCq27Gh received CONNACK (0)
Client mosq/E7sCzVC1BFNFCq27Gh sending PUBLISH (d0, q0, r0, m1, '/comm_robot/xperiahello_01/attrs', ... (47 bytes))
Client mosq/E7sCzVC1BFNFCq27Gh sending DISCONNECT
```

when executing the above command, show below messages on the opened terminal.

example)
```
Client mosq/A8qOmsuUfYwYuMDNBr received PUBLISH (d0, q0, r0, m0, '/comm_robot/xperiahello_01/attrs', ... (48 bytes))
2019-05-11T10:28:01.1557538081+09:00|move|dest01
```

### confirm comm_robot entity

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${COMM_ROBOT_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/${COMM_ROBOT_ID}/ | jq .

example)
```json
{
  "id": "xperiahello_01",
  "type": "comm_robot",
  "TimeInstant": {
    "type": "ISO8601",
    "value": "2019-05-11T10:28:01.1557538081+09:00",
    "metadata": {}
  },
  "move": {
    "type": "string",
    "value": "dest01",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-05-11T10:28:01.1557538081+09:00"
      }
    }
  }
}
```

### confirm cygnus-mongo

In [ ]:
kubectl exec mongodb-0 -c mongodb-replicaset -- mongo sth_${FIWARE_SERVICE} --eval "db.getCollection(\"sth_${COMM_ROBOT_SERVICEPATH}_${COMM_ROBOT_ID}_${COMM_ROBOT_TYPE}\").find().sort({recvTime: -1})"

example)
```
MongoDB shell version v4.1.10
connecting to: mongodb://127.0.0.1:27017/sth_guide_demo?compressors=disabled&gssapiServiceName=mongodb
Implicit session: session { "id" : UUID("5c2be38e-a546-452a-8147-1055f735f198") }
MongoDB server version: 4.1.10
{ "_id" : ObjectId("5cd613e7130da800118c559e"), "recvTime" : ISODate("2019-05-11T00:14:29.155Z"), "attrName" : "move", "attrType" : "string", "attrValue" : "dest01" }
```

## test mobile_robot device

### prepare a command to subscribe all topics

In [ ]:
echo "mosquitto_sub -h mqtt.${DOMAIN} -p 8883 --cafile ${CORE_ROOT}/secrets/DST_Root_CA_X3.pem -d -u iotagent -P ${MQTT__iotagent} -t /#"

### subscribe all topic

_Outside of this notebook_
1. open a ternminal.
1. run the above command displayed `prepare a command to subscribe all topics`.

### update the move attribute of mobile_robot to emulate sending the command

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -i -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${MOBILE_ROBOT_SERVICEPATH}" -H "Content-Type: application/json" https://api.${DOMAIN}/orion/v2/entities/${MOBILE_ROBOT_ID}/attrs?type=${MOBILE_ROBOT_TYPE} -X PATCH -d @-<<__EOS__
{
  "move": {
    "value": "up"
  }
}
__EOS__

example)
```
HTTP/1.1 204 No Content
content-length: 0
fiware-correlator: 84b12a5e-738d-11e9-83e5-46b1c9ec8ccd
date: Sat, 11 May 2019 01:38:46 GMT
x-envoy-upstream-service-time: 71
server: envoy
```

when executing the above command, show below messages on the opened terminal.

example)
```
Client mosq/gZ1wcOVoAWYDodg8I6 received PUBLISH (d0, q0, r0, m0, '/mobile_robot/rosbot2_01/cmd', ... (18 bytes))
rosbot2_01@move|up
```

### confirm mobile_robot entity

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${MOBILE_ROBOT_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/${MOBILE_ROBOT_ID}/ | jq .

example)
```json
{
  "id": "rosbot2_01",
  "type": "mobile_robot",
  "TimeInstant": {
    "type": "ISO8601",
    "value": "2019-05-11T01:38:46.00Z",
    "metadata": {}
  },
  "capacity": {
    "type": "float32",
    "value": " ",
    "metadata": {}
  },
  "charge": {
    "type": "float32",
    "value": " ",
    "metadata": {}
  },
  "current": {
    "type": "float32",
    "value": " ",
    "metadata": {}
  },
  "design_capacity": {
    "type": "float32",
    "value": " ",
    "metadata": {}
  },
  "move_info": {
    "type": "commandResult",
    "value": " ",
    "metadata": {}
  },
  "move_status": {
    "type": "commandStatus",
    "value": "PENDING",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-05-11T01:38:46.591Z"
      }
    }
  },
  "percentage": {
    "type": "float32",
    "value": " ",
    "metadata": {}
  },
  "r_mode": {
    "type": "string",
    "value": " ",
    "metadata": {}
  },
  "r_state": {
    "type": "string",
    "value": " ",
    "metadata": {}
  },
  "theta": {
    "type": "float32",
    "value": " ",
    "metadata": {}
  },
  "voltage": {
    "type": "float32",
    "value": " ",
    "metadata": {}
  },
  "x": {
    "type": "float32",
    "value": " ",
    "metadata": {}
  },
  "y": {
    "type": "float32",
    "value": " ",
    "metadata": {}
  },
  "move": {
    "type": "string",
    "value": "",
    "metadata": {}
  }
}
```

### publish a MQTT message to emulate receiving the command result from mobile_robot

In [ ]:
mosquitto_pub -h mqtt.${DOMAIN} -p 8883 --cafile ${CORE_ROOT}/secrets/DST_Root_CA_X3.pem -d -u iotagent -P ${MQTT__iotagent} -t /${MOBILE_ROBOT_TYPE}/${MOBILE_ROBOT_ID}/cmdexe -m "${MOBILE_ROBOT_ID}@move|received up"

example)
```
Client mosq/zmmwCxWFQSP0u6XAFN sending CONNECT
Client mosq/zmmwCxWFQSP0u6XAFN received CONNACK (0)
Client mosq/zmmwCxWFQSP0u6XAFN sending PUBLISH (d0, q0, r0, m1, '/mobile_robot/rosbot2_01/cmdexe', ... (27 bytes))
Client mosq/zmmwCxWFQSP0u6XAFN sending DISCONNECT
```

when executing the above command, show below messages on the opened terminal.

example)
```
Client mosq/gZ1wcOVoAWYDodg8I6 received PUBLISH (d0, q0, r0, m0, '/mobile_robot/rosbot2_01/cmdexe', ... (27 bytes))
rosbot2_01@move|received up
```

### confirm mobile_robot entity

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${MOBILE_ROBOT_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/${MOBILE_ROBOT_ID}/ | jq .

example)
```json
{
  "id": "rosbot2_01",
  "type": "mobile_robot",
  "TimeInstant": {
    "type": "ISO8601",
    "value": "2019-05-11T01:46:43.00Z",
    "metadata": {}
  },
  "capacity": {
    "type": "float32",
    "value": " ",
    "metadata": {}
  },
  "charge": {
    "type": "float32",
    "value": " ",
    "metadata": {}
  },
  "current": {
    "type": "float32",
    "value": " ",
    "metadata": {}
  },
  "design_capacity": {
    "type": "float32",
    "value": " ",
    "metadata": {}
  },
  "move_info": {
    "type": "commandResult",
    "value": "received up",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-05-11T01:46:43.407Z"
      }
    }
  },
  "move_status": {
    "type": "commandStatus",
    "value": "OK",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-05-11T01:46:43.407Z"
      }
    }
  },
  "percentage": {
    "type": "float32",
    "value": " ",
    "metadata": {}
  },
  "r_mode": {
    "type": "string",
    "value": " ",
    "metadata": {}
  },
  "r_state": {
    "type": "string",
    "value": " ",
    "metadata": {}
  },
  "theta": {
    "type": "float32",
    "value": " ",
    "metadata": {}
  },
  "voltage": {
    "type": "float32",
    "value": " ",
    "metadata": {}
  },
  "x": {
    "type": "float32",
    "value": " ",
    "metadata": {}
  },
  "y": {
    "type": "float32",
    "value": " ",
    "metadata": {}
  },
  "move": {
    "type": "string",
    "value": "",
    "metadata": {}
  }
}
```

### confirm cygnus-mongo

In [ ]:
kubectl exec mongodb-0 -c mongodb-replicaset -- mongo sth_${FIWARE_SERVICE} --eval "db.getCollection(\"sth_${MOBILE_ROBOT_SERVICEPATH}_${MOBILE_ROBOT_ID}_${MOBILE_ROBOT_TYPE}\").find().sort({recvTime: -1})"

example)
```
MongoDB shell version v4.1.10
connecting to: mongodb://127.0.0.1:27017/sth_guide_demo?compressors=disabled&gssapiServiceName=mongodb
Implicit session: session { "id" : UUID("e60b18fb-76ee-4955-9249-60ed30f1fb66") }
MongoDB server version: 4.1.10
{ "_id" : ObjectId("5cd6298641f12a0011c4e5a4"), "recvTime" : ISODate("2019-05-11T01:46:43.407Z"), "attrName" : "move_status", "attrType" : "commandStatus", "attrValue" : "OK" }
{ "_id" : ObjectId("5cd6298641f12a0011c4e5a5"), "recvTime" : ISODate("2019-05-11T01:46:43.407Z"), "attrName" : "move_info", "attrType" : "commandResult", "attrValue" : "received up" }
{ "_id" : ObjectId("5cd627a641f12a0011c4e5a3"), "recvTime" : ISODate("2019-05-11T01:38:46.591Z"), "attrName" : "move_status", "attrType" : "commandStatus", "attrValue" : "PENDING" }
{ "_id" : ObjectId("5cd620e441f12a0011c4e5a0"), "recvTime" : ISODate("2019-05-11T01:09:53.842Z"), "attrName" : "move_status", "attrType" : "commandStatus", "attrValue" : "UNKNOWN" }
{ "_id" : ObjectId("5cd61943130da800118c559f"), "recvTime" : ISODate("2019-05-11T00:37:20.750Z"), "attrName" : "move_status", "attrType" : "commandStatus", "attrValue" : "UNKNOWN" }
```

### publish a MQTT message to emulate receiving the internal state of mobile_robot

In [ ]:
d=$(date '+%Y-%m-%dT%H:%M:%S.%s+09:00')
mosquitto_pub -h mqtt.${DOMAIN} -p 8883 --cafile ${CORE_ROOT}/secrets/DST_Root_CA_X3.pem -d -u iotagent -P ${MQTT__iotagent} -t /${MOBILE_ROBOT_TYPE}/${MOBILE_ROBOT_ID}/attrs -m "${d}|r_mode|navi"

example)
```
Client mosq/2ykUaqktpEWPLuk0sv sending CONNECT
Client mosq/2ykUaqktpEWPLuk0sv received CONNACK (0)
Client mosq/2ykUaqktpEWPLuk0sv sending PUBLISH (d0, q0, r0, m1, '/mobile_robot/rosbot2_01/attrs', ... (48 bytes))
Client mosq/2ykUaqktpEWPLuk0sv sending DISCONNECT
```

when executing the above command, show below messages on the opened terminal.

example)
```
Client mosq/gZ1wcOVoAWYDodg8I6 received PUBLISH (d0, q0, r0, m0, '/mobile_robot/rosbot2_01/attrs', ... (48 bytes))
2019-05-11T10:57:10.1557539830+09:00|r_mode|navi
```

### confirm mobile_robot entity

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${MOBILE_ROBOT_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/${MOBILE_ROBOT_ID}/ | jq .

example)
```json
{
  "id": "rosbot2_01",
  "type": "mobile_robot",
  "TimeInstant": {
    "type": "ISO8601",
    "value": "2019-05-11T10:57:10.1557539830+09:00",
    "metadata": {}
  },
  "capacity": {
    "type": "float32",
    "value": " ",
    "metadata": {}
  },
  "charge": {
    "type": "float32",
    "value": " ",
    "metadata": {}
  },
  "current": {
    "type": "float32",
    "value": " ",
    "metadata": {}
  },
  "design_capacity": {
    "type": "float32",
    "value": " ",
    "metadata": {}
  },
  "move_info": {
    "type": "commandResult",
    "value": "received up",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-05-11T01:46:43.407Z"
      }
    }
  },
  "move_status": {
    "type": "commandStatus",
    "value": "OK",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-05-11T01:46:43.407Z"
      }
    }
  },
  "percentage": {
    "type": "float32",
    "value": " ",
    "metadata": {}
  },
  "r_mode": {
    "type": "string",
    "value": "navi",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-05-11T10:57:10.1557539830+09:00"
      }
    }
  },
  "r_state": {
    "type": "string",
    "value": " ",
    "metadata": {}
  },
  "theta": {
    "type": "float32",
    "value": " ",
    "metadata": {}
  },
  "voltage": {
    "type": "float32",
    "value": " ",
    "metadata": {}
  },
  "x": {
    "type": "float32",
    "value": " ",
    "metadata": {}
  },
  "y": {
    "type": "float32",
    "value": " ",
    "metadata": {}
  },
  "move": {
    "type": "string",
    "value": "",
    "metadata": {}
  }
}
```

### confirm cygnus-mongo

In [ ]:
kubectl exec mongodb-0 -c mongodb-replicaset -- mongo sth_${FIWARE_SERVICE} --eval "db.getCollection(\"sth_${MOBILE_ROBOT_SERVICEPATH}_${MOBILE_ROBOT_ID}_${MOBILE_ROBOT_TYPE}\").find().sort({recvTime: -1})"

example)
```
MongoDB shell version v4.1.10
connecting to: mongodb://127.0.0.1:27017/sth_guide_demo?compressors=disabled&gssapiServiceName=mongodb
Implicit session: session { "id" : UUID("292767da-6c76-4f5a-9bee-3a86861b46b5") }
MongoDB server version: 4.1.10
{ "_id" : ObjectId("5cd62bf541f12a0011c4e5a6"), "recvTime" : ISODate("2019-05-11T01:57:10.155Z"), "attrName" : "r_mode", "attrType" : "string", "attrValue" : "Navi" }
...
```

### publish a MQTT message to emulate receiving the position of mobile_robot

In [ ]:
d=$(date '+%Y-%m-%dT%H:%M:%S.%s+09:00')
mosquitto_pub -h mqtt.${DOMAIN} -p 8883 --cafile ${CORE_ROOT}/secrets/DST_Root_CA_X3.pem -d -u iotagent -P ${MQTT__iotagent} -t /${MOBILE_ROBOT_TYPE}/${MOBILE_ROBOT_ID}/attrs -m "${d}|x|1.05|y|0.01|theta|0.05"

example)
```
Client mosq/itDkLV5rOf9bbMdgSq sending CONNECT
Client mosq/itDkLV5rOf9bbMdgSq received CONNACK (0)
Client mosq/itDkLV5rOf9bbMdgSq sending PUBLISH (d0, q0, r0, m1, '/mobile_robot/rosbot2_01/attrs', ... (61 bytes))
Client mosq/itDkLV5rOf9bbMdgSq sending DISCONNECT
```

when executing the above command, show below messages on the opened terminal.

example)
```
Client mosq/gZ1wcOVoAWYDodg8I6 received PUBLISH (d0, q0, r0, m0, '/mobile_robot/rosbot2_01/attrs', ... (61 bytes))
2019-05-11T11:19:53.1557541193+09:00|x|1.05|y|0.01|theta|0.05
```

### confirm mobile_robot entity

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${MOBILE_ROBOT_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/${MOBILE_ROBOT_ID}/ | jq .

example)
```json
{
  "id": "rosbot2_01",
  "type": "mobile_robot",
  "TimeInstant": {
    "type": "ISO8601",
    "value": "2019-05-11T11:19:53.1557541193+09:00",
    "metadata": {}
  },
  "capacity": {
    "type": "float32",
    "value": " ",
    "metadata": {}
  },
  "charge": {
    "type": "float32",
    "value": " ",
    "metadata": {}
  },
  "current": {
    "type": "float32",
    "value": " ",
    "metadata": {}
  },
  "design_capacity": {
    "type": "float32",
    "value": " ",
    "metadata": {}
  },
  "move_info": {
    "type": "commandResult",
    "value": "received up",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-05-11T01:46:43.407Z"
      }
    }
  },
  "move_status": {
    "type": "commandStatus",
    "value": "OK",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-05-11T01:46:43.407Z"
      }
    }
  },
  "percentage": {
    "type": "float32",
    "value": " ",
    "metadata": {}
  },
  "r_mode": {
    "type": "string",
    "value": "Navi",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-05-11T10:57:10.1557539830+09:00"
      }
    }
  },
  "r_state": {
    "type": "string",
    "value": " ",
    "metadata": {}
  },
  "theta": {
    "type": "float32",
    "value": "0.05",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-05-11T11:19:53.1557541193+09:00"
      }
    }
  },
  "voltage": {
    "type": "float32",
    "value": " ",
    "metadata": {}
  },
  "x": {
    "type": "float32",
    "value": "1.05",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-05-11T11:19:53.1557541193+09:00"
      }
    }
  },
  "y": {
    "type": "float32",
    "value": "0.01",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-05-11T11:19:53.1557541193+09:00"
      }
    }
  },
  "move": {
    "type": "string",
    "value": "",
    "metadata": {}
  }
}
```

### confirm cygnus-mongo

In [ ]:
kubectl exec mongodb-0 -c mongodb-replicaset -- mongo sth_${FIWARE_SERVICE} --eval "db.getCollection(\"sth_${MOBILE_ROBOT_SERVICEPATH}_${MOBILE_ROBOT_ID}_${MOBILE_ROBOT_TYPE}\").find().sort({recvTime: -1})"

example)
```
MongoDB shell version v4.1.10
connecting to: mongodb://127.0.0.1:27017/sth_guide_demo?compressors=disabled&gssapiServiceName=mongodb
Implicit session: session { "id" : UUID("488d73df-3ae2-4602-965d-1c84a8d2f6e1") }
MongoDB server version: 4.1.10
{ "_id" : ObjectId("5cd6314a130da800118c55a1"), "recvTime" : ISODate("2019-05-11T02:19:53.155Z"), "attrName" : "x", "attrType" : "float32", "attrValue" : "1.05" }
{ "_id" : ObjectId("5cd6314a130da800118c55a2"), "recvTime" : ISODate("2019-05-11T02:19:53.155Z"), "attrName" : "y", "attrType" : "float32", "attrValue" : "0.01" }
{ "_id" : ObjectId("5cd6314a130da800118c55a3"), "recvTime" : ISODate("2019-05-11T02:19:53.155Z"), "attrName" : "theta", "attrType" : "float32", "attrValue" : "0.05" }
...
```

### publish a MQTT message to emulate receiving the battery state of mobile_robot

In [ ]:
d=$(date '+%Y-%m-%dT%H:%M:%S.%s+09:00')
mosquitto_pub -h mqtt.${DOMAIN} -p 8883 --cafile ${CORE_ROOT}/secrets/DST_Root_CA_X3.pem -d -u iotagent -P ${MQTT__iotagent} -t /${MOBILE_ROBOT_TYPE}/${MOBILE_ROBOT_ID}/attrs -m "${d}|voltage|11.73|current|0.25|charge|0.0|capacity|0.0|design_capacity|0.0|percentage|95.0"

example)
```
Client mosq/9c5dtJqZt4CN0ZuPUg sending CONNECT
Client mosq/9c5dtJqZt4CN0ZuPUg received CONNACK (0)
Client mosq/9c5dtJqZt4CN0ZuPUg sending PUBLISH (d0, q0, r0, m1, '/mobile_robot/rosbot2_01/attrs', ... (123 bytes))
Client mosq/9c5dtJqZt4CN0ZuPUg sending DISCONNECT
```

when executing the above command, show below messages on the opened terminal.

example)
```
Client mosq/gZ1wcOVoAWYDodg8I6 received PUBLISH (d0, q0, r0, m0, '/mobile_robot/rosbot2_01/attrs', ... (123 bytes))
2019-05-11T11:23:13.1557541393+09:00|voltage|11.73|current|0.25|charge|0.0|capacity|0.0|design_capacity|0.0|percentage|95.0
```

### confirm mobile_robot entity

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${MOBILE_ROBOT_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/${MOBILE_ROBOT_ID}/ | jq .

example)
```json
{
  "id": "rosbot2_01",
  "type": "mobile_robot",
  "TimeInstant": {
    "type": "ISO8601",
    "value": "2019-05-11T11:23:13.1557541393+09:00",
    "metadata": {}
  },
  "capacity": {
    "type": "float32",
    "value": "0.0",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-05-11T11:23:13.1557541393+09:00"
      }
    }
  },
  "charge": {
    "type": "float32",
    "value": "0.0",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-05-11T11:23:13.1557541393+09:00"
      }
    }
  },
  "current": {
    "type": "float32",
    "value": "0.25",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-05-11T11:23:13.1557541393+09:00"
      }
    }
  },
  "design_capacity": {
    "type": "float32",
    "value": "0.0",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-05-11T11:23:13.1557541393+09:00"
      }
    }
  },
  "move_info": {
    "type": "commandResult",
    "value": "received up",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-05-11T01:46:43.407Z"
      }
    }
  },
  "move_status": {
    "type": "commandStatus",
    "value": "OK",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-05-11T01:46:43.407Z"
      }
    }
  },
  "percentage": {
    "type": "float32",
    "value": "95.0",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-05-11T11:23:13.1557541393+09:00"
      }
    }
  },
  "r_mode": {
    "type": "string",
    "value": "Navi",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-05-11T10:57:10.1557539830+09:00"
      }
    }
  },
  "r_state": {
    "type": "string",
    "value": " ",
    "metadata": {}
  },
  "theta": {
    "type": "float32",
    "value": "0.05",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-05-11T11:19:53.1557541193+09:00"
      }
    }
  },
  "voltage": {
    "type": "float32",
    "value": "11.73",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-05-11T11:23:13.1557541393+09:00"
      }
    }
  },
  "x": {
    "type": "float32",
    "value": "1.05",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-05-11T11:19:53.1557541193+09:00"
      }
    }
  },
  "y": {
    "type": "float32",
    "value": "0.01",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-05-11T11:19:53.1557541193+09:00"
      }
    }
  },
  "move": {
    "type": "string",
    "value": "",
    "metadata": {}
  }
}
```

### confirm cygnus-mongo

In [ ]:
kubectl exec mongodb-0 -c mongodb-replicaset -- mongo sth_${FIWARE_SERVICE} --eval "db.getCollection(\"sth_${MOBILE_ROBOT_SERVICEPATH}_${MOBILE_ROBOT_ID}_${MOBILE_ROBOT_TYPE}\").find().sort({recvTime: -1})"

example)
```
MongoDB shell version v4.1.10
connecting to: mongodb://127.0.0.1:27017/sth_guide_demo?compressors=disabled&gssapiServiceName=mongodb
Implicit session: session { "id" : UUID("7e4acdfe-c515-447e-9bbe-9bcc198163c9") }
MongoDB server version: 4.1.10
{ "_id" : ObjectId("5cd6321141f12a0011c4e5af"), "recvTime" : ISODate("2019-05-11T02:23:13.155Z"), "attrName" : "voltage", "attrType" : "float32", "attrValue" : "11.73" }
{ "_id" : ObjectId("5cd6321141f12a0011c4e5b0"), "recvTime" : ISODate("2019-05-11T02:23:13.155Z"), "attrName" : "current", "attrType" : "float32", "attrValue" : "0.25" }
{ "_id" : ObjectId("5cd6321141f12a0011c4e5b1"), "recvTime" : ISODate("2019-05-11T02:23:13.155Z"), "attrName" : "charge", "attrType" : "float32", "attrValue" : "0.0" }
{ "_id" : ObjectId("5cd6321141f12a0011c4e5b2"), "recvTime" : ISODate("2019-05-11T02:23:13.155Z"), "attrName" : "capacity", "attrType" : "float32", "attrValue" : "0.0" }
{ "_id" : ObjectId("5cd6321141f12a0011c4e5b3"), "recvTime" : ISODate("2019-05-11T02:23:13.155Z"), "attrName" : "design_capacity", "attrType" : "float32", "attrValue" : "0.0" }
{ "_id" : ObjectId("5cd6321141f12a0011c4e5b4"), "recvTime" : ISODate("2019-05-11T02:23:13.155Z"), "attrName" : "percentage", "attrType" : "float32", "attrValue" : "95.0" }
...
```

## test dest_button device

### prepare a command to subscribe all topics

In [ ]:
echo "mosquitto_sub -h mqtt.${DOMAIN} -p 8883 --cafile ${CORE_ROOT}/secrets/DST_Root_CA_X3.pem -d -u iotagent -P ${MQTT__iotagent} -t /#"

### subscribe all topic

_Outside of this notebook_
1. open a ternminal.
1. run the above command displayed `prepare a command to subscribe all topics`.

### publish a MQTT message to emulate receiving the event from dest_button

In [ ]:
d=$(date '+%Y-%m-%dT%H:%M:%S.%s+09:00')
mosquitto_pub -h mqtt.${DOMAIN} -p 8883 --cafile ${CORE_ROOT}/secrets/DST_Root_CA_X3.pem -d -u iotagent -P ${MQTT__iotagent} -t /${DEST_BUTTON_TYPE}/${DEST_BUTTON_ID}/attrs -m "${d}|push|${d}"

example)
```
Client mosq/Ngwy6XHyACGR6ToMuD sending CONNECT
Client mosq/Ngwy6XHyACGR6ToMuD received CONNACK (0)
Client mosq/Ngwy6XHyACGR6ToMuD sending PUBLISH (d0, q0, r0, m1, '/button/button_01/attrs', ... (78 bytes))
Client mosq/Ngwy6XHyACGR6ToMuD sending DISCONNECT
```

when executing the above command, show below messages on the opened terminal.

example)
```
Client mosq/gZ1wcOVoAWYDodg8I6 received PUBLISH (d0, q0, r0, m0, '/button/button_01/attrs', ... (78 bytes))
2019-05-11T11:48:01.1557542881+09:00|push|2019-05-11T11:48:01.1557542881+09:00
```

### confirm dest_button entity

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DEST_BUTTON_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/${DEST_BUTTON_ID}/ | jq .

example)
```json
{
  "id": "button_01",
  "type": "button",
  "TimeInstant": {
    "type": "ISO8601",
    "value": "2019-05-11T11:48:01.1557542881+09:00",
    "metadata": {}
  },
  "push": {
    "type": "string",
    "value": "2019-05-11T11:48:01.1557542881+09:00",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-05-11T11:48:01.1557542881+09:00"
      }
    }
  }
}
```

### confirm cygnus-mongo

In [ ]:
kubectl exec mongodb-0 -c mongodb-replicaset -- mongo sth_${FIWARE_SERVICE} --eval "db.getCollection(\"sth_${DEST_BUTTON_SERVICEPATH}_${DEST_BUTTON_ID}_${DEST_BUTTON_TYPE}\").find().sort({recvTime: -1})"

example)
```
MongoDB shell version v4.1.10
connecting to: mongodb://127.0.0.1:27017/sth_guide_demo?compressors=disabled&gssapiServiceName=mongodb
Implicit session: session { "id" : UUID("0aaec8d8-0131-4fcd-b38b-bc24b1f16f75") }
MongoDB server version: 4.1.10
{ "_id" : ObjectId("5cd637e1130da800118c55a6"), "recvTime" : ISODate("2019-05-11T02:48:01.155Z"), "attrName" : "push", "attrType" : "string", "attrValue" : "2019-05-11T11:48:01.1557542881+09:00" }
```

## test dest_led device

### prepare a command to subscribe all topics

In [ ]:
echo "mosquitto_sub -h mqtt.${DOMAIN} -p 8883 --cafile ${CORE_ROOT}/secrets/DST_Root_CA_X3.pem -d -u iotagent -P ${MQTT__iotagent} -t /#"

### subscribe all topic

_Outside of this notebook_
1. open a ternminal.
1. run the above command displayed `prepare a command to subscribe all topics`.

### update the action attribute of dest_led to emulate sending the command

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -i -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DEST_LED_SERVICEPATH}" -H "Content-Type: application/json" https://api.${DOMAIN}/orion/v2/entities/${DEST_LED_ID}/attrs?type=${DEST_LED_TYPE} -X PATCH -d @-<<__EOS__
{
  "action": {
    "value": "on"
  }
}
__EOS__

example)
```
HTTP/1.1 204 No Content
content-length: 0
fiware-correlator: 19ab5ec4-739b-11e9-a0fb-f2a8c31f77d6
date: Sat, 11 May 2019 03:15:59 GMT
x-envoy-upstream-service-time: 34
server: envoy
```

when executing the above command, show below messages on the opened terminal.

example)
```
Client mosq/gZ1wcOVoAWYDodg8I6 received PUBLISH (d0, q0, r0, m0, '/led/led_01/cmd', ... (16 bytes))
led_01@action|on
```

### confirm mobile_robot entity

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DEST_LED_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/${DEST_LED_ID}/ | jq .

example)
```json
{
  "id": "led_01",
  "type": "led",
  "TimeInstant": {
    "type": "ISO8601",
    "value": "2019-05-11T03:15:59.00Z",
    "metadata": {}
  },
  "action_info": {
    "type": "commandResult",
    "value": " ",
    "metadata": {}
  },
  "action_status": {
    "type": "commandStatus",
    "value": "PENDING",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-05-11T03:15:59.953Z"
      }
    }
  },
  "action": {
    "type": "string",
    "value": "",
    "metadata": {}
  }
}
```

### publish a MQTT message to emulate receiving the command result from dest_led

In [ ]:
mosquitto_pub -h mqtt.${DOMAIN} -p 8883 --cafile ${CORE_ROOT}/secrets/DST_Root_CA_X3.pem -d -u iotagent -P ${MQTT__iotagent} -t /${DEST_LED_TYPE}/${DEST_LED_ID}/cmdexe -m "${DEST_LED_ID}@action|received on"

example)
```
Client mosq/9tqyQYJG7l2Iy2Gbuj sending CONNECT
Client mosq/9tqyQYJG7l2Iy2Gbuj received CONNACK (0)
Client mosq/9tqyQYJG7l2Iy2Gbuj sending PUBLISH (d0, q0, r0, m1, '/led/led_01/cmdexe', ... (25 bytes))
Client mosq/9tqyQYJG7l2Iy2Gbuj sending DISCONNECT
```

when executing the above command, show below messages on the opened terminal.

example)
```
Client mosq/gZ1wcOVoAWYDodg8I6 received PUBLISH (d0, q0, r0, m0, '/led/led_01/cmdexe', ... (25 bytes))
led_01@action|received on
```

### confirm dest_led entity

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DEST_LED_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/${DEST_LED_ID}/ | jq .

example)
```json
{
  "id": "led_01",
  "type": "led",
  "TimeInstant": {
    "type": "ISO8601",
    "value": "2019-05-11T03:18:45.00Z",
    "metadata": {}
  },
  "action_info": {
    "type": "commandResult",
    "value": "received on",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-05-11T03:18:45.304Z"
      }
    }
  },
  "action_status": {
    "type": "commandStatus",
    "value": "OK",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-05-11T03:18:45.304Z"
      }
    }
  },
  "action": {
    "type": "string",
    "value": "",
    "metadata": {}
  }
}
```

### confirm cygnus-mongo

In [ ]:
kubectl exec mongodb-0 -c mongodb-replicaset -- mongo sth_${FIWARE_SERVICE} --eval "db.getCollection(\"sth_${DEST_LED_SERVICEPATH}_${DEST_LED_ID}_${DEST_LED_TYPE}\").find().sort({recvTime: -1})"

example)
```
MongoDB shell version v4.1.10
connecting to: mongodb://127.0.0.1:27017/sth_guide_demo?compressors=disabled&gssapiServiceName=mongodb
Implicit session: session { "id" : UUID("39e9ac5c-96b0-481b-8c08-67b15487b944") }
MongoDB server version: 4.1.10
{ "_id" : ObjectId("5cd63f16130da800118c55a7"), "recvTime" : ISODate("2019-05-11T03:18:45.304Z"), "attrName" : "action_info", "attrType" : "commandResult", "attrValue" : "received on" }
{ "_id" : ObjectId("5cd63f16130da800118c55a8"), "recvTime" : ISODate("2019-05-11T03:18:45.304Z"), "attrName" : "action_status", "attrType" : "commandStatus", "attrValue" : "OK" }
{ "_id" : ObjectId("5cd63e6f41f12a0011c4e5b5"), "recvTime" : ISODate("2019-05-11T03:15:59.953Z"), "attrName" : "action_status", "attrType" : "commandStatus", "attrValue" : "PENDING" }
{ "_id" : ObjectId("5cd63ae15046420012c5ce53"), "recvTime" : ISODate("2019-05-11T03:00:49.386Z"), "attrName" : "action_status", "attrType" : "commandStatus", "attrValue" : "UNKNOWN" }
```